# Velocity gradients and divergence in gas structure

We calculate the velocity gradients $\nabla$v of the velocity field of each molecule. This will tell us about local motions within the fibers. For this, we use the velocity_tools package (Pineda et al.)

We also compare the velocity gradients with the polarization vectors to see if the kinematics are influenced by magnetic fields.

For this, first we need the velocity gradient of HC$_3$N and N$_2$H$^+$ $\nabla$v in km s$^{-1}$ pc$^{-1}$, in their native resolutions and in a common one. we modify the resolution of HC$_3$N for this. From 'clustering/comparison_velocities_N2Hp_HC3N.ipynb' we have the clusters velocities in the same pixel grid.



In [1]:
import numpy as np
import os
from astropy.io import fits
import astropy.units as u
import matplotlib.pyplot as plt
from astropy.wcs import WCS
from matplotlib import cm
from astropy.coordinates import SkyCoord
import pandas as pd
import sys
sys.path.append('/home/mvaldivi/velocity_tools/')
from velocity_tools import vgrad

%matplotlib widget

In [3]:
# gaussian fit results

gaussfitfolder = '../bayes_frame/nested-sampling/'
hc3n1gfile = gaussfitfolder + 'HC3N/NGC1333-SE-mle-x1_filtered_QA.fits'
hc3n2gfile = gaussfitfolder + 'HC3N/NGC1333-SE-mle-x2_filtered_QA.fits'
hc3n3gfile = gaussfitfolder + 'HC3N/NGC1333-SE-mle-x3_filtered_QA.fits'
n2hp1gfile = gaussfitfolder + 'N2Hp/NGC1333-SE-mle-x1_filtered_QA.fits'
n2hp2gfile = gaussfitfolder + 'N2Hp/NGC1333-SE-mle-x2_filtered_QA.fits'

gaussifitfiles = np.array([hc3n1gfile, hc3n2gfile, hc3n3gfile, n2hp1gfile, n2hp2gfile])

# per fiber
clusterfolder = '../clustering/'
hc3nbluefile = clusterfolder + 'clusters_blue_HC3N.fits'
hc3nredfile = clusterfolder + 'clusters_red_HC3N.fits'
n2hpbluefile = clusterfolder + 'cluster_blue_HDBSCAN_N2Hp.fits'
n2hpredfile = clusterfolder + 'cluster_red_HDBSCAN_N2Hp.fits'

fiberfiles = np.array([hc3nbluefile, hc3nredfile, n2hpbluefile, n2hpredfile])

# per cluster
# HC3N has 8 clusters

hc3nclusterbase = clusterfolder + 'cluster{}_HDBSCAN_HC3N.fits'
hc3nclusterfiles = [hc3nclusterbase.format(i) for i in range(7)]

headertodel = ['crval3', 'cdelt3', 'cunit3', 'ctype3', 'crpix3', 'naxis3']

overwrite=False

# we need to save the velocity in a separate file to use vgrad

In [4]:
def separate_vel_comp(params, parheader):
    newhead = parheader.copy()
    newhead['naxis'] = 2
    newhead['wcsaxes'] = 2
    for flag in headertodel:
        del newhead[flag]
    if parheader["NAXIS3"] == 6:
        velfield = params[1]
        velerror = params[4]
        
    elif parheader["NAXIS3"] == 12:
        velfield = [params[1], params[4]]
        velerror = [params[7], params[10]]
        # newhead['naxis'] = 3
        # newhead['wcsaxes'] = 3
        # newhead['naxis3'] = 2
    elif parheader["NAXIS3"] == 18:
        velfield = [params[1], params[4], params[7]]
        velerror = [params[10], params[13], params[16]]
        # newhead['naxis'] = 3
        # newhead['wcsaxes'] = 3
        # newhead['naxis3'] = 3
    return velfield, velerror, newhead

def separate_vel_file(paramsfile):
    params, parheader = fits.getdata(paramsfile, header=True)
    velfield, velerror, newhead = separate_vel_comp(params, parheader)
    if len(np.shape(velfield)) > 2:
        for i in range(np.shape(velfield)[0]):
            fits.writeto(paramsfile[:-5]+'_{}_vel.fits'.format(i+1), velfield[i], newhead, overwrite=True)
            fits.writeto(paramsfile[:-5]+'_{}_vel_unc.fits'.format(i+1), velerror[i], newhead, overwrite=True)
        
    else:    
        fits.writeto(paramsfile[:-5]+'_vel.fits', velfield, newhead, overwrite=True)
        fits.writeto(paramsfile[:-5]+'_vel_unc.fits', velerror, newhead, overwrite=True)
        
    

In [5]:
# we separate the velocity to save it apart in the cases needed

filelist = np.concatenate([gaussifitfiles, fiberfiles, hc3nclusterfiles])

for file in filelist:
    if not os.path.exists(file[:-5]+'_vel.fits') or not os.path.exists(file[:-5]+'_vel_unc.fits') or overwrite:
        print('Runing file {}'.format(file))
        separate_vel_file(file)
    else:
        print('File {} already exists and will not be overwritten'.format(file[:-5]+'_vel.fits'))
              

# del newhead[]

File ../bayes_frame/nested-sampling/HC3N/NGC1333-SE-mle-x1_filtered_QA_vel.fits already exists and will not be overwritten
File ../bayes_frame/nested-sampling/HC3N/NGC1333-SE-mle-x2_filtered_QA_vel.fits already exists and will not be overwritten
File ../bayes_frame/nested-sampling/HC3N/NGC1333-SE-mle-x3_filtered_QA_vel.fits already exists and will not be overwritten
File ../bayes_frame/nested-sampling/N2Hp/NGC1333-SE-mle-x1_filtered_QA_vel.fits already exists and will not be overwritten
File ../bayes_frame/nested-sampling/N2Hp/NGC1333-SE-mle-x2_filtered_QA_vel.fits already exists and will not be overwritten
File ../clustering/clusters_blue_HC3N_vel.fits already exists and will not be overwritten
File ../clustering/clusters_red_HC3N_vel.fits already exists and will not be overwritten
File ../clustering/cluster_blue_HDBSCAN_N2Hp_vel.fits already exists and will not be overwritten
File ../clustering/cluster_red_HDBSCAN_N2Hp_vel.fits already exists and will not be overwritten
File ../clust

Now that we separated the velocity from the files, we calculate the gradient in each case

In [6]:
def gradient_calc_save(velfile, velerrorfile, savefile, nbeams=2, distance=298*u.pc):
    velheader = fits.getheader(velfile)

    beammaj, beammin, pixwidth = (velheader['BMAJ'], velheader['BMIN'], velheader['CDELT2']) * u.deg
    equivbeam = np.sqrt(beammaj * beammin) # equivalent circular beam
    samplingwidth = equivbeam * nbeams  # 2 beams, we use this as a radius
    print('The radius of the calculation area is {}'.format(np.round(samplingwidth.to(u.arcsec), 2)))
    # the minimum number of pixels available is given in nyquist sampling
    result_all = vgrad.vfit_image(velfile, velerrorfile,
                              distance=298*u.pc, width = samplingwidth.to(u.arcsec))
    grad_mag = result_all['grad']
    grad_mag_err = result_all['grad_err']
    gradmaghead = result_all['header'].copy()
    gradmaghead['BUNIT'] = 'km s-1 pc-1'
    gradmaghead['NAXIS'] = 3
    gradmaghead['NAXIS3'] = 2
    fits.writeto(savefile+'_magnitude.fits', [grad_mag, grad_mag_err], gradmaghead, overwrite=True)
    grad_PA = result_all['posang']
    grad_PA_err = result_all['paerr']
    PAhead = gradmaghead.copy()
    PAhead['BUNIT'] = 'deg'
    fits.writeto(savefile+'_PA.fits', [grad_PA, grad_PA_err], gradmaghead, overwrite=True)
    grad_vc = result_all['vc'].value
    vchead = result_all['header'].copy()
    fits.writeto(savefile+'_vc.fits', grad_vc, vchead, overwrite=True)
    

In [11]:
mle_extra_grad_files = [gaussfitfolder + s for s in ['HC3N/NGC1333-SE-mle-x2_filtered_QA_1.fits', 'HC3N/NGC1333-SE-mle-x2_filtered_QA_2.fits',
                        'HC3N/NGC1333-SE-mle-x3_filtered_QA_1.fits', 'HC3N/NGC1333-SE-mle-x3_filtered_QA_2.fits',
                        'HC3N/NGC1333-SE-mle-x3_filtered_QA_3.fits', 'N2Hp/NGC1333-SE-mle-x2_filtered_QA_1.fits',
                        'N2Hp/NGC1333-SE-mle-x2_filtered_QA_2.fits']]

filelist_new = np.concatenate([gaussifitfiles, mle_extra_grad_files, fiberfiles, hc3nclusterfiles])

for file in filelist_new:
    savefile = file[:-5]+'_vel_grad'
    if not os.path.exists(savefile+'_magnitude.fits') or overwrite:
        print('Running gradient calculation for {}'.format(file))
        velfile = file[:-5]+'_vel.fits'
        velfile_unc = file[:-5]+'_vel_unc.fits'
        gradient_calc_save(velfile, velfile_unc, savefile) #if we run the function it will save
    else:
        print('Gradient calculation for file {} already exists and will not be overwritten'.format(file))

Gradient calculation for file ../bayes_frame/nested-sampling/HC3N/NGC1333-SE-mle-x1_filtered_QA.fits already exists and will not be overwritten
Gradient calculation for file ../bayes_frame/nested-sampling/HC3N/NGC1333-SE-mle-x2_filtered_QA.fits already exists and will not be overwritten
Gradient calculation for file ../bayes_frame/nested-sampling/HC3N/NGC1333-SE-mle-x3_filtered_QA.fits already exists and will not be overwritten
Gradient calculation for file ../bayes_frame/nested-sampling/N2Hp/NGC1333-SE-mle-x1_filtered_QA.fits already exists and will not be overwritten
Gradient calculation for file ../bayes_frame/nested-sampling/N2Hp/NGC1333-SE-mle-x2_filtered_QA.fits already exists and will not be overwritten
Running gradient calculation for ../bayes_frame/nested-sampling/HC3N/NGC1333-SE-mle-x2_filtered_QA_1.fits
The radius of the calculation area is 9.58 arcsec
Running gradient calculation for ../bayes_frame/nested-sampling/HC3N/NGC1333-SE-mle-x2_filtered_QA_2.fits
The radius of the 

In [9]:
mle_extra_grad_files

'HC3N/NGC1333-SE-mle-x2_filtered_QA_1.fits../bayes_frame/nested-sampling/HC3N/NGC1333-SE-mle-x2_filtered_QA_2.fits../bayes_frame/nested-sampling/HC3N/NGC1333-SE-mle-x3_filtered_QA_1.fits../bayes_frame/nested-sampling/HC3N/NGC1333-SE-mle-x3_filtered_QA_2.fits../bayes_frame/nested-sampling/HC3N/NGC1333-SE-mle-x3_filtered_QA_3.fits../bayes_frame/nested-sampling/N2Hp/NGC1333-SE-mle-x2_filtered_QA_1.fits../bayes_frame/nested-sampling/N2Hp/NGC1333-SE-mle-x2_filtered_QA_2.fits'